# Add computational predictions to data

This step is the most complicated step of processing a dataset for calculating OpenKnotScore. The OpenKnotScore metric relies on an ensemble of candidate secondary structures created by various structure prediction algorithms. There are many ways to generate those secondary structures, but in this notebook, we will focus on how to generate those structures efficiently using the predictors available through `arnie` and running on Stanford's Sherlock computing cluster.

Generally, we've been processing large sequence libraries for OpenKnotScore calculation. On Sherlock, the best way to process a large dataset is in parallel, splitting the dataset into subsets and processing each on a different node using SLURM's batch array feature. The `1b.SplitDataForSherlockProcessing.ipynb` notebook provides a simple way to generate these subset files (again, tailored for use on Sherlock). 

Picking the right size of subset is important. How many sequences you should attempt to process on each node depends on how large the overall dataset is and which prediction algorithms you are using (some algorithms take much longer to run, so smaller subsets will be more effective). See
https://www.sherlock.stanford.edu/docs/advanced-topics/job-management/ and https://www.sherlock.stanford.edu/docs/user-guide/running-jobs/ for more details.

## Using the scripts
The scripts we will be running are available in the `scripts` folder in this repo. `GeneratePredictions.py` is the actual job that will be run on each node, and `get_predictions.sbatch` is the SLURM submission script that we will use to request all our jobs be run. One source of difficulty is correctly setting up your Sherlock environment so that arnie can actually access all the algorithms you want to use.

## GeneratePredictions.py
This script takes an input dataframe with sequence and reactivity data, and generates predictions using the selected prediction algorithms. There's only one important change you need to make in this file:

**Update the `predictors` and `heuristic_predictions` variables with the predictors you actually want to run.** 

For example, if you don't want to run `pknots`, set `"pknots": False` in the `predictors` dictionary.

## get_predictions.sbatch
This script submits your jobs to the Sherlock job queue. We need to make several updates to this file.
- Set `#SBATCH --job-name=` to the job name you want to use. **Make sure this matches the job name you used when generating subset data files.** We use this job name to set the path to the data directory in the GeneratePredictions.py script
- Set `#SBATCH --mail-user=` to your email so you can receive email notifications of the progress of your submission
- Set `#SBATCH --array=` to the number of subset files in your data directory (e.g. 1-1000). Note that there are job submission limts you might run into if you try to submit with too large an array here.
- You should also update the resources requests (`--time`,`--partition`,`--cpus-per-task`,`--mem`). https://www.sherlock.stanford.edu/docs/user-guide/running-jobs/#resource-requests has helpful tips on how to estimate your job's resource usage. This particular job can be difficult to estimate, because arnie shells out prediction to downstream programs that spin up their own processes, threads, etc.